In [92]:
import pandas as pd
import numpy as np
import os
import  nltk
import random


In [96]:
# df_sentiment = pd.read_csv("./sentiment_gop_cleaned.csv", engine='python')
df_sentiment = pd.read_csv("./cleaned_airline_sentiment.csv", engine='python')
# df_sentiment.drop(['Unnamed: 0'], axis=1, inplace=True)
# print(df_gop.shape)
df_sentiment.head()

,tweet_id,tweet_created,sentiment,full_text,cleaned_text
0,570301130888122368,2015-02-24 11:15:59 -0800,non-negative,@VirginAmerica plus you've added commercials to the experience... tacky.,plus you have added commercials to the experience tacky
1,570301083672813571,2015-02-24 11:15:48 -0800,non-negative,@VirginAmerica I didn't today... Must mean I need to take another trip!,i did not today must mean i need to take another trip
2,570301031407624196,2015-02-24 11:15:36 -0800,negative,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &...",it is really aggressive to blast obnoxious entertainment in your guests faces they have little ...
3,570300817074462722,2015-02-24 11:14:45 -0800,negative,@VirginAmerica and it's a really big bad thing about it,and it is a really big bad thing about it
4,570300767074181121,2015-02-24 11:14:33 -0800,negative,@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's r...,seriously would pay $30 a flight for seats that did not have this playing it is really the only ...


In [97]:
df_sentiment['sentiment'].value_counts()

negative        8822
non-negative    4402
Name: sentiment, dtype: int64

In [49]:
# df_sentiment = df_sentiment.sort_values(by=['sentiment'])
# df_sentiment.reset_index(drop=True)
# df_sentiment= df_sentiment.iloc[3000:]
# df_sentiment = df_sentiment.sample(frac=1).reset_index(drop=True)
# df_sentiment['sentiment'].value_counts()

Negative    5170
Neutral     2861
Positive    2149
Name: sentiment, dtype: int64

In [95]:
# df_sentiment['sentiment'] = pd.Categorical(df_sentiment['sentiment'], ["Neutral", "Negative", "Positive"])
# df_sentiment = df_sentiment.sort_values(by=['sentiment'])
# df_sentiment.reset_index(drop=True)
# df_sentiment= df_sentiment.iloc[700:]
# df_sentiment = df_sentiment.sample(frac=1).reset_index(drop=True)
# df_sentiment['sentiment'].value_counts()
def change_vals(row):
    if row == 'Neutral' or row == 'Positive':
        return "Non-Negative"
    else:
        return row
df_sentiment['sentiment'] = df_sentiment['sentiment'].apply(change_vals)

In [98]:
data = df_sentiment
data['sentiment'].value_counts()

negative        8822
non-negative    4402
Name: sentiment, dtype: int64

In [111]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
# from sklearn.feature_extraction.text import CountVectorizer
# from keras.preprocessing.text import Tokenizer
# from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
# from keras.callbacks import EarlyStopping
# from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

max_len = 130
num_labels = 2
vocab_size = 4000
batch_size = 32
num_epochs = 10
embed_dim = 128
lstm_out = 196

# train_size = int(len(data) * .8)
# train_text = data['cleaned_text'][:train_size]
# train_tags = data['sentiment'][:train_size]
# print(train_tags.value_counts())
# test_text = data['cleaned_text'][train_size:]
# test_tags = data['sentiment'][train_size:]

# tokenizer = Tokenizer(num_words=vocab_size)
# tokenizer.fit_on_texts(train_text)


tokenizer = Tokenizer(num_words=vocab_size, split=' ')
tokenizer.fit_on_texts(data['cleaned_text'].values)
X = tokenizer.texts_to_sequences(data['cleaned_text'].values)
X = pad_sequences(X)
Y = pd.get_dummies(data['sentiment']).values

print(len(data['sentiment'].values))
arr_y = np.array(Y)

sanity_check = {'text': data['cleaned_text'].values, 'sentiment': data['sentiment'].values, 'negative':arr_y[:,0], 'non-negative': arr_y[:,1]}
sanity_check = pd.DataFrame(data=sanity_check)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 42)

# x_train = tokenizer.texts_to_matrix(train_text, mode='tfidf')
# x_test = tokenizer.texts_to_matrix(test_text, mode='tfidf')

encoder = LabelBinarizer()
encoder.fit(Y)

# y_train = train_tags
# y_test = test_tags


13224


LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [112]:
sanity_check.sample(50)

,text,sentiment,negative,non-negative
3841,i am not this person but i have been sitting in denver for three hours and all i want to do is g...,negative,1,0
2495,is really getting horrible so much for tucking in my kids tonight delta transfers milesstatus ri...,negative,1,0
10432,45 minutes means no public transit home and an expensive cab promises does not make up for it,negative,1,0
13218,right on cue with the delays:ok_hand:,negative,1,0
7458,tickets were confirmed and sent out find out today that they cancelled flightled our tickets mon...,negative,1,0
11650,finally at final destination but not wo extra significant travel expenses missed work how can y...,negative,1,0
11124,please can you tell me why dfw has a weather advisory and rdu does not the entire state of nc ha...,non-negative,0,1
3786,your customer service for mileage plus customers is a disgrace i just spent 45 min on the phone ...,negative,1,0
393,can you help with flights to get a 2y old battling cancer who needs treatment in :united_states:...,non-negative,0,1
4254,yeah they are somewhere hopefully getting them back today just frustrated tweeting,negative,1,0


In [101]:
X_train.shape, Y_train.shape

((8860, 89), (8860, 2))

In [102]:
print(sum(Y_test))

[130 138]


In [103]:
# #let us build a basic model
# model = Sequential()
# model.add(Dense(512, input_shape=(vocab_size,)))
# model.add(Activation('relu'))
# model.add(Dropout(0.3))
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.3))
# model.add(Dense(num_labels))
# model.add(Activation('softmax'))
# model.summary()

# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

model = Sequential()
model.add(Embedding(vocab_size, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_labels,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 89, 128)           512000    
_________________________________________________________________
spatial_dropout1d_8 (Spatial (None, 89, 128)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 394       
Total params: 767,194
Trainable params: 767,194
Non-trainable params: 0
_________________________________________________________________
None


In [104]:
# num_epochs = 10
# batch_size = 128
# history = model.fit(X_train, Y_train,
#                     batch_size=batch_size,
#                     epochs=num_epochs,
#                     verbose=2,
#                     validation_split=0.2)

model.fit(X_train, Y_train, epochs = num_epochs, batch_size=batch_size, verbose = 2)

Epoch 1/10
 - 35s - loss: 0.4730 - accuracy: 0.7783
Epoch 2/10
 - 36s - loss: 0.3264 - accuracy: 0.8567
Epoch 3/10
 - 32s - loss: 0.2741 - accuracy: 0.8839
Epoch 4/10
 - 27s - loss: 0.2332 - accuracy: 0.9040
Epoch 5/10
 - 30s - loss: 0.2055 - accuracy: 0.9151
Epoch 6/10
 - 33s - loss: 0.1720 - accuracy: 0.9301
Epoch 7/10
 - 32s - loss: 0.1434 - accuracy: 0.9404
Epoch 8/10
 - 33s - loss: 0.1334 - accuracy: 0.9476
Epoch 9/10
 - 33s - loss: 0.1136 - accuracy: 0.9577
Epoch 10/10
 - 32s - loss: 0.1032 - accuracy: 0.9593


In [105]:
score, acc = model.evaluate(X_test, Y_test,
                       batch_size=batch_size, verbose=2)

print('Test accuracy:', acc)

Test accuracy: 0.819890022277832


In [106]:
non_cnt, neg_cnt, non_correct, neg_correct = 0, 0, 0, 0
for i in range(len(X_test)):
    
    result = model.predict(X_test[i].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]

    # correct prediction
    if np.argmax(result) == np.argmax(Y_test[i]):
        if np.argmax(Y_test[i]) == 0:
            neg_correct += 1
        else:
            non_correct += 1
            
    # label count   
    if np.argmax(Y_test[i]) == 0:
        neg_cnt += 1
    else:
        non_cnt += 1

print("non-negative_acc", non_correct/non_cnt*100, "%")
print("negative_acc", neg_correct/neg_cnt*100, "%")



non-negative_acc 73.06064880112835 %
negative_acc 86.28649015614393 %


In [25]:
text_labels = encoder.classes_
res_dict = { 'predicted': [], 'actual': []}
for i in range(len(X_test)):
    prediction = model.predict(np.array([X_test[i]]))
    predicted_label = text_labels[np.argmax(prediction[0])]
#     print(predicted_label)
    # print(test_files_names.iloc[i])
    # res_dict['text'].append(test_text.iloc[i])
    res_dict['predicted'].append(predicted_label)
    if(Y_test[i][0] == 1):
        res_dict['actual'].append('Neutral')
    elif Y_test[i][1] == 1 :
        res_dict['actual'].append('Negative')
    else:
        res_dict['actual'].append('Positive')
    


df_res = pd.DataFrame(data=res_dict)

print(df_res['predicted'].value_counts())
print(df_res['actual'].value_counts())

[1 0 0]
[0 0 1]
[0 1 0]
[0 1 0]
[1 0 0]
[0 0 1]
[0 0 1]
[1 0 0]
[1 0 0]
[1 0 0]
[0 0 1]
[1 0 0]
[0 1 0]
[0 0 1]
[0 1 0]
[0 1 0]
[0 0 1]
[1 0 0]
[0 0 1]
[0 0 1]
[1 0 0]
[1 0 0]
[0 0 1]
[0 1 0]
[0 1 0]
[1 0 0]
[1 0 0]
[0 1 0]
[1 0 0]
[0 0 1]
[1 0 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 0 1]
[0 0 1]
[0 0 1]
[0 1 0]
[0 0 1]
[0 1 0]
[0 0 1]
[0 0 1]
[1 0 0]
[1 0 0]
[0 0 1]
[0 1 0]
[0 0 1]
[1 0 0]
[1 0 0]
[0 1 0]
[1 0 0]
[0 0 1]
[1 0 0]
[0 0 1]
[0 1 0]
[0 1 0]
[0 1 0]
[0 0 1]
[0 0 1]
[1 0 0]
[0 0 1]
[1 0 0]
[0 0 1]
[0 0 1]
[0 0 1]
[1 0 0]
[0 0 1]
[0 1 0]
[0 0 1]
[0 1 0]
[0 0 1]
[1 0 0]
[0 0 1]
[1 0 0]
[1 0 0]
[1 0 0]
[0 0 1]
[1 0 0]
[0 0 1]
[0 0 1]
[0 0 1]
[0 1 0]
[0 1 0]
[1 0 0]
[1 0 0]
[1 0 0]
[1 0 0]
[0 0 1]
[0 0 1]
[0 0 1]
[1 0 0]
[0 0 1]
[0 0 1]
[1 0 0]
[0 0 1]
[0 1 0]
[0 1 0]
[0 0 1]
[1 0 0]
[0 0 1]
[0 1 0]
[1 0 0]
[0 0 1]
[0 1 0]
[0 1 0]
[0 0 1]
[0 0 1]
[1 0 0]
[1 0 0]
[0 0 1]
[1 0 0]
[0 1 0]
[0 1 0]
[1 0 0]
[0 1 0]
[0 1 0]
[1 0 0]
[1 0 0]
[0 1 0]
[0 0 1]
[0 0 1]
[0 0 1]
[1 0 0]
[1 0 0]


[1 0 0]
[0 1 0]
[0 0 1]
[0 1 0]
[0 1 0]
[1 0 0]
[0 1 0]
[0 1 0]
[0 0 1]
[0 0 1]
[1 0 0]
[0 0 1]
[0 1 0]
[0 1 0]
[1 0 0]
[0 0 1]
[1 0 0]
[0 0 1]
[0 0 1]
[0 1 0]
[0 1 0]
[0 1 0]
[0 0 1]
[1 0 0]
[0 0 1]
[0 0 1]
[0 1 0]
[0 0 1]
[0 1 0]
[0 0 1]
[1 0 0]
[0 0 1]
[0 1 0]
[1 0 0]
[1 0 0]
[0 0 1]
[0 0 1]
[0 0 1]
[1 0 0]
[0 1 0]
[1 0 0]
[0 0 1]
[1 0 0]
[1 0 0]
[0 0 1]
[1 0 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 0 1]
[1 0 0]
[0 1 0]
[0 0 1]
[1 0 0]
[0 1 0]
[1 0 0]
[0 0 1]
[1 0 0]
[0 0 1]
[1 0 0]
[0 0 1]
[0 1 0]
[1 0 0]
[1 0 0]
[1 0 0]
[0 1 0]
[0 0 1]
[1 0 0]
[0 0 1]
[1 0 0]
[0 0 1]
[1 0 0]
[0 1 0]
[1 0 0]
[1 0 0]
[1 0 0]
[0 1 0]
[0 0 1]
[1 0 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[0 1 0]
[1 0 0]
[0 0 1]
[0 0 1]
[0 1 0]
[1 0 0]
[0 1 0]
[0 0 1]
[0 0 1]
[0 0 1]
[0 1 0]
[0 1 0]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[1 0 0]
[1 0 0]
[0 1 0]
[1 0 0]
[1 0 0]
[1 0 0]
[1 0 0]
[0 1 0]
[0 0 1]
[0 0 1]
[1 0 0]
[0 0 1]
[1 0 0]
[1 0 0]
[0 1 0]
[0 0 1]
[1 0 0]
[0 0 1]
[0 0 1]
[1 0 0]
[1 0 0]
[1 0 0]
[1 0 0]
[0 1 0]
[0 1 0]


[0 1 0]
[1 0 0]
[0 0 1]
[0 1 0]
[0 1 0]
[0 0 1]
[0 1 0]
[1 0 0]
[0 1 0]
[1 0 0]
[0 0 1]
[0 1 0]
[0 0 1]
[0 0 1]
[0 1 0]
[0 0 1]
[0 1 0]
[0 0 1]
[0 1 0]
[0 0 1]
[0 1 0]
[0 0 1]
[0 0 1]
[1 0 0]
[0 0 1]
[1 0 0]
[0 0 1]
[0 1 0]
[1 0 0]
[1 0 0]
[1 0 0]
[0 1 0]
[0 0 1]
[0 1 0]
[1 0 0]
[1 0 0]
[1 0 0]
[0 0 1]
[0 0 1]
[1 0 0]
[0 1 0]
[1 0 0]
[0 0 1]
[1 0 0]
[0 1 0]
[0 0 1]
[1 0 0]
[0 1 0]
[1 0 0]
[0 0 1]
[1 0 0]
[1 0 0]
[0 1 0]
2    824
1    692
0    616
Name: predicted, dtype: int64
Positive    720
Neutral     713
Negative    699
Name: actual, dtype: int64


In [26]:
pd.options.display.max_colwidth = 100

df_res.head(50)

,predicted,actual
0,1,Neutral
1,2,Positive
2,2,Negative
3,2,Negative
4,0,Neutral
5,2,Positive
6,2,Positive
7,0,Neutral
8,0,Neutral
9,1,Neutral


## Clustering

In [109]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans


model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors)

13224


In [110]:
print(X[0])
print(X[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0  528    4   15
 1160 2391    1    3  181]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   2  71   9  89 722 549   2  66   1 138 136 175]
